<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/pytorch/pythorch_roberta_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# CHANGES:
## lr increase
## TRAIN_BATCH_SIZE
## VALID_BATCH_SIZE

In [2]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [0]:
# TODOS:
## add tic and toc to calculate the total time!
## check the pretrained model
## remove neutral from the filtering

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Installs PyTorch, PyTorch/XLA, and Torchvision
# Copy this cell into your own notebooks to use PyTorch on Cloud TPUs 
# Warning: this may take a couple minutes to run

import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

VERSION = "20200325"  # @ param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3727  100  3727    0     0  23006      0 --:--:-- --:--:-- --:--:-- 22865
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Uninstalling torch-1.5.0a0+d6149a7:
  Successfully uninstalled torch-1.5.0a0+d6149a7
Uninstalling torchvision-0.6.0a0+3c254fb:
  Successfully uninstalled torchvision-0.6.0a0+3c254fb
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
| [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...
- [1 files][114.5 MiB/114.5 MiB]                                                
Operation completed over 1 objects/114.5 MiB.               

In [0]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl


In [7]:
!pip install transformers
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time

In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
CHECKPOINT_PATH_TORCH = INPUT_PATH + "model_checkpoint/roberta/torch/"
# ROBERTA_PATH = INPUT_PATH + 'models/output_twitter/'
ROBERTA_PATH = INPUT_PATH + 'models/roberta-base/'

VER='tpu-v1'

In [0]:
def print_to_file(text):
  try:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print_to_file('[%s]' % self.name,)
        print_to_file('Elapsed: %s' % (time.time() - self.tstart))

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

In [0]:
class config():
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 64
    VALID_BATCH_SIZE = 32
    EPOCHS = 3
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
        vocab_file=f"{ROBERTA_PATH}/vocab.json", 
        merges_file=f"{ROBERTA_PATH}/merges.txt", 
        lowercase=True,
        add_prefix_space=True
    )
    LR = 4e-5

In [0]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [0]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [0]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [0]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

In [0]:
train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().reset_index()#.sample(n=2000).reset_index()

df_test = pd.read_csv(DATA_PATH + 'test.csv')
df_test.loc[:, "selected_text"] = df_test.text.values

In [0]:
fold_ids = []
num_folds = 5

kfold = model_selection.KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold_num, (train_idx, valid_idx) in enumerate(kfold.split(train_df.text)):
    fold_ids.append((train_idx, valid_idx))

In [0]:

def map_fn(index, flags):
  ## Setup 
    fold_num = flags["fold_num"]
    # Sets a common random seed - both for initialization and ensuring graph is the same
    torch.manual_seed(6666)

    # Acquires the (unique) Cloud TPU core corresponding to this process's index
    device = xm.xla_device()  
    print_to_file("TPU in use:" + str(xm.xla_real_devices([str(device)])[0]))


    ## Dataloader construction
    
    train_idx = fold_ids[fold_num][0]
    valid_idx = fold_ids[fold_num][1]
    train_df_fold = train_df.loc[train_idx]
    valid_df_fold = train_df.loc[valid_idx]

    num_train_batches_per_core = int(train_df_fold.shape[0]/config.TRAIN_BATCH_SIZE/8)
    num_valid_batches_per_core = int(valid_df_fold.shape[0]/config.VALID_BATCH_SIZE/8)

    train_dataset = TweetDataset(
        tweet=train_df_fold.text.values,
        sentiment=train_df_fold.sentiment.values,
        selected_text=train_df_fold.selected_text.values
    )

    valid_dataset = TweetDataset(
        tweet=valid_df_fold.text.values,
        sentiment=valid_df_fold.sentiment.values,
        selected_text=valid_df_fold.selected_text.values
    )

    train_sampler = torch.utils.data.distributed.DistributedSampler(
    train_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)

    test_sampler = torch.utils.data.distributed.DistributedSampler(
    valid_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)

    # Creates dataloaders, which load data in batches
    # Note: test loader is not shuffled or sampled
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        sampler=train_sampler,
        num_workers=8,
        drop_last=True)

    test_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        sampler=test_sampler,
        shuffle=False,
        num_workers=8,
        drop_last=True)


  ## Network, optimizer, and loss function creation

  #  Note: each process has its own identical copy of the model
  #  Even though each model is created independently, they're also
  #  created in the same way.

    model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
    model_config.output_hidden_states = True
    model = TweetModel(conf=model_config).to(device).train()

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LR)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=int(train_df_fold.shape[0] / config.TRAIN_BATCH_SIZE * config.EPOCHS) #TODO: shall we divide by 8TPUs
    )

    ## Trains
    train_start = time.time()
    for epoch in range(config.EPOCHS):
        if xm.is_master_ordinal():
            print("starting epoch %i on core %i" % (epoch, index))
        para_train_loader = pl.ParallelLoader(train_loader, [device]).per_device_loader(device)

        if xm.is_master_ordinal():
            batch_set = tqdm(para_train_loader, total=num_train_batches_per_core, position=0, leave=True)
        else: 
            batch_set = para_train_loader
        for batch_num, batch in enumerate(batch_set):
            ids = batch["ids"]
            token_type_ids = batch["token_type_ids"]
            mask = batch["mask"]
            targets_start = batch["targets_start"]
            targets_end = batch["targets_end"]
            sentiment = batch["sentiment"]
            orig_selected = batch["orig_selected"]
            orig_tweet = batch["orig_tweet"]
            targets_start = batch["targets_start"]
            targets_end = batch["targets_end"]
            offsets = batch["offsets"]

            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            optimizer.zero_grad()
            loss.backward()
            xm.optimizer_step(optimizer)
            scheduler.step()

    elapsed_train_time = time.time() - train_start
    print_to_file("TPU %i - finished training. Train time was: %.2fmin" % (index, elapsed_train_time/60)) 


    ## Evaluation
    model.eval()
    eval_start = time.time()
    with torch.no_grad():

        final_output = []
        jaccard_scores = []
        para_test_loader = pl.ParallelLoader(test_loader, [device]).per_device_loader(device)
        if xm.is_master_ordinal():
            batch_set = tqdm(para_test_loader, total=num_valid_batches_per_core, position=0, leave=True)
        else: 
            batch_set = para_test_loader
        for batch_num, batch in enumerate(batch_set):
            outputs_start, outputs_end = model(
                ids=batch["ids"],
                mask=batch["mask"],
                token_type_ids=batch["token_type_ids"]
            )
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

            for px in range(len(batch["offsets"])):

                targets_start = batch["targets_start"] [px]
                targets_end = batch["targets_end"][px]
                offsets = batch["offsets"][px]        
                
                jac, output_sentence = calculate_jaccard_score(
                    original_tweet=batch["orig_tweet"][px],
                    target_string=batch["orig_selected"][px],
                    sentiment_val=batch["sentiment"][px],
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=batch["offsets"][px]
                )
                final_output.append(output_sentence)
                jaccard_scores.append(jac)
        jacc_total[fold_num][index][2] = np.mean(jaccard_scores)
    xm.save(model.state_dict(), CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,fold_num), master_only=True)
    elapsed_eval_time = time.time() - eval_start
    print_to_file("TPU %i - Evaluation time was: %.2fmin, jaccard average: %.5f" % (index, elapsed_eval_time/60, np.mean(jaccard_scores)))

In [0]:
import warnings
warnings.filterwarnings('ignore')
# Configures training (and evaluation) parameters

jacc_total = {}
for fold_num in range(5):
    jacc_total[fold_num] = {}
    for tpu_core in range(8):
        jacc_total[fold_num][tpu_core] = {}
        for epoch_num in range(8):
            jacc_total[fold_num][tpu_core][epoch_num] = 0
            
for fold_num in range(5):
    print_to_file("Starting Training for Fold %i"%fold_num)
    flags = {"fold_num": fold_num}
    xmp.spawn(map_fn, args=(flags,), nprocs=8, start_method='fork')

Starting Training for Fold 0
TPU in use:TPU:0
TPU in use:TPU:4
TPU in use:TPU:1
TPU in use:TPU:7
TPU in use:TPU:5
TPU in use:TPU:6
TPU in use:TPU:3
TPU in use:TPU:2
starting epoch 0 on core 0


100%|██████████| 42/42 [01:31<00:00,  2.17s/it]


starting epoch 1 on core 0


100%|██████████| 42/42 [00:31<00:00,  1.31it/s]


starting epoch 2 on core 0


...

100%|██████████| 42/42 [00:31<00:00,  1.33it/s]

TPU 6 - finished training. Train time was: 1.64min
TPU 7 - finished training. Train time was: 1.64min
TPU 1 - finished training. Train time was: 1.69min
TPU 2 - finished training. Train time was: 1.64min
TPU 3 - finished training. Train time was: 1.62min
TPU 4 - finished training. Train time was: 1.64min
TPU 5 - finished training. Train time was: 1.65min


100%|██████████| 42/42 [00:31<00:00,  1.31it/s]


TPU 0 - finished training. Train time was: 1.70min


  0%|          | 0/21 [00:00<?, ?it/s]